<a href="https://colab.research.google.com/github/Tikquuss/electre_tri/blob/main/Electre_tri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Les fonctions utiles**

**Les imports nécessaires**

In [1]:
import itertools
import os
import pandas as pd

## **Étape 1 : Détermination des indices de concordance partiels**

### **Calcul de $c_j(H, b_i)$ et de $c_j(b_i, H)$**

Pour chaque critère $j$, l’indice de concordance partiel entre l’aliment $H$ et le profil bi est donné par :
* Si la fonction $g_j$ est à maximiser :

$
c_j(H, b_i) = \left\{
    \begin{array}{ll}
        1 & \mbox{si } g_j(H) \geq  g_j(b_i) \\
        0 & \mbox{sinon.}
    \end{array}
\right. 
$ et $
c_j(b_i, H) = \left\{
    \begin{array}{ll}
        1 & \mbox{si } g_j(b_i) \geq  g_j(H) \\
        0 & \mbox{sinon.}
    \end{array}
\right.
$
* Si la fonction $g_j$ est à minimiser :

$
c_j(H, b_i) = \left\{
    \begin{array}{ll}
        1 & \mbox{si } g_j(b_i) \geq  g_j(H) \\
        0 & \mbox{sinon.}
    \end{array}
\right. 
$ et $
c_j(b_i, H) = \left\{
    \begin{array}{ll}
        1 & \mbox{si } g_j(H) \geq  g_j(b_i) \\
        0 & \mbox{sinon.}
    \end{array}
\right.
$

avec $g_j(H)$ et $g_j(b_i)$ représentant respectivement le score de $H$ et $b_i$ sur le critère $j$. Dans notre cas, ce score représente l’évaluation qualitative attribuée à $H$ et $b_i$. Par conséquent, la comparaison $g_j(H) \geq g_j(b_i)$ signifie simplement que la
valeur qualitative de $H$ est au moins aussi bonne que celle de $b_i$.

In [7]:
def get_c_j(H : list, b_i : list, j : int, type_critere : str) :
  """retourne cj(H,b_i) et cj(b_i,H) selon la formule indiquée ci-dessus, tous des apparténant à {0, 1}
    Parametres :
      - H (list or array) : aliment (tableau de taille égale au nombre de critères, exemple [1, 10, 100, 2.1, 0] pur 5 critères)
      - b_i (list or array) : profil (meme commentaire que pour H, exemple : [2, 1, 1, 100, 1, 1] )
      - j (int) : indice du critère (entre 0 et le nombre de critères)
      - type_critere (str) : chaine de caractère indiquant si le critère j est à maximiser (max) ou à minimiser (min)
  """
  assert type_critere in ["max", "min"]
  if type_critere == "max" :
    c_j_H_b_i = 1 if H[j] >= b_i[j] else 0
    c_j_b_i_H = 1 if b_i[j] >= H[j] else 0
  if type_critere == "min" :
    c_j_H_b_i = 1 if b_i[j] >= H[j] else 0
    c_j_b_i_H = 1 if H[j] >= b_i[j] else 0
  
  return c_j_H_b_i, c_j_b_i_H

### **Fonction de calcul des indices de concordances partiels**

In [9]:
def get_indices_de_concordance_partiels(criteres : list, aliments : dict, profils : dict) :
  """Retourne un dictionnaire conténant pour chaque aliment H et chaque critère b_i
     les indices de concordance partiels c(H, bi) et c(bi, H), tous des réelle (float)

     Parametres :
        - criteres (list) : la liste des critères et de leur types
              Exemple : [["Énergie", 'min'], ["Acide Gras sat.", 'min'], ["Sucre", 'min'], ["Sodium", 'min'], ["Protéine", 'max'], ["Fibre", 'max']]
        - aliments (dict) : dictionnaire conténant les aliments (la clé correspond au nom de l'aliment, et la valeur aux 
                            évaluation qualitative attribuée à l'aliment pour chaque critères)
              Exemple : aliments = {"aliment1" : [1, 2, 3, 1, 0.1, 10], 'aliment2': [2, 0, 1, 5, 1, 10]}
        - profils (dict) : dictionnaire conténant les profils (la clé correspond au nom du profil, et la valeur aux 
                            évaluation qualitative attribuée au profil pour chaque critères)
              Exemple : profils = {"b6" : [100, 0, 0, 0, 100, 100], "b5" : [1550, 11, 0.8, 0.3, 10, 11], 
                                  "b4" : [1650, 14, 1, 0.4, 7, 8], "b3" : [1750, 17, 1.7, 0.5, 4, 5], 
                                  "b2" : [1850, 20, 4, 0.6, 3, 2.5], "b1" : [10000, 100, 100, 100, 0, 0]}
  """
  c = {}
  for j in range(len(criteres)) :
    c[j] = {}
    for H, b_i in itertools.product(*[aliments.keys(), profils.keys()]) :
      c[j][H] = c[j].get(H, {})
      c[j][b_i] = c[j].get(b_i, {})
      c[j][H][b_i], c[j][b_i][H] = get_c_j(H = aliments[H], b_i = profils[b_i], j = j, type_critere = criteres[j][1])
  return c

## **Étape 2 : Détermination des indices de concordance globaux**

L’indice de concordance global entre l’aliment $H$ et le profil $b_i$ est donné par la formule suivante :

$
C(H, b_i) = \frac{\sum_{j=1}^{n} k_jc_j(H,b_i)}{\sum_{j=1}^{n} k_j} 
$
et
$
C(b_i, H) = \frac{\sum_{j=1}^{n} k_jc_j(b_i, H)}{\sum_{j=1}^{n} k_j} 
$

où $k_j$ est le poids du critère $j$ et $n$ le nombre de critères

In [11]:
def get_indices_de_concordance_globaux(n : int, indices_de_concordance_partiels : dict, proids : list) :
  """Retourne un dictionnaire conténant pour chaque aliment H et chaque critère b_i
     les indices de concordance globaux c(H, bi) et c(bi, H), tous des réelle (float)

     Parametres :
        - n (list) : nombre de critères
        - indices_de_concordance_partiels (dict) : dictionnaires conténant les indices de concordance partiels calculé comme illustrer à l'étapes 1
        - proids (liste) : listes de poids pour chaque critères (poids[j] = poids du critère j)
              Exemple : poids = [1, 1, 1, 1, 2, 2]
  """
  C = {}
  sum_k = sum(proids)
  for H, b_i in itertools.product(*[aliments.keys(), profils.keys()]) :
    C[H] = C.get(H, {})
    C[b_i] = C.get(b_i, {})
    C[H][b_i] = sum([ proids[j] * indices_de_concordance_partiels[j][H][b_i] for j in range(n)]) / sum_k 
    C[b_i][H] = sum([ proids[j] * indices_de_concordance_partiels[j][b_i][H] for j in range(n)]) / sum_k 
  return C

## **Étape 3 : Détermination de la relation de surclassement S**

La relation de surclassement se définie à l’aide de l’indice de coupe $\lambda$, appelé seuil de majorité (en général supérieur à 50%), qui représente le paramètre déterminant la situation de préférence entre l’aliment $H$ et le profil $b_i$. Ainsi pour l’aliment $H$ et un profil $b_i$ :
- $H$ surclasse $b_i$ et on notera $H S b_i$ si et seulement si $C(H, b_i) \geq \lambda$.
- $b_i$ surclasse $H$ et on notera $b_i S H$ si et seulement si $C(b_i, H) \geq \lambda$.

In [12]:
def surclass(seuil_de_majorite : float, H : str, b_i : str, indices_de_concordance_globaux : dict):
	"""relation de surclassement : retourne H S b_i et b_i S H (qui sont tous de boolean : c'est à dire Vrai(True) ou Faux(False))
     Parametres :
        - seuil_de_majorite (float) : réel entre 0 et 1 (ou 0% à 100%) répresentant le seuil de majorité
        - H (str) : nom de l'aliment
        - b_i (str) : nom du profil
        - indices_de_concordance_globaux (dict) : dictionnaires conténant les indices de concordance globaux calculer comme illustrer à l'étapes 2
  """
	H_S_b_i = indices_de_concordance_globaux[H][b_i] >= seuil_de_majorite
	b_i_S_H = indices_de_concordance_globaux[b_i][H] >= seuil_de_majorite
	return H_S_b_i, b_i_S_H

## **Étape 4 : Procédures d’affectation**

Deux procédures d’affectation de l’aliment H sont possibles :

- **Procédure pessimiste** : Pour chaque aliment $H$, faire décroitre les indices des profils de $r$ jusqu’au premier indice $k$ tel que $H S b_k$. L’aliment $H$ est alors affecté à la catégorie $C_k$.

  Si $H S b_k$ n’est jamais réalisée, alors $H$ est affectée à la catégorie la moins bonne, $C_1$ **(Olivier Sobri et al., page 5/24)** [1]
- **Procédure optimiste** : pour chaque aliment $H$, faire croitre les indices des profils de $1$ jusqu’au premier indice $k$ tel que $b_k S H$ et $non(H S b_k)$. L’aliment $H$ est alors affecté à la catégorie $C_{k−1}$.

  Si $b_k S H$ et $non(H S b_k)$ n’est jamais réalisée, alors $H$ est affectée à la catégorie la meilleure, $C_r$ **(Olivier Sobri et al., page 5/24)** [1]

[1] Intégration de la méthode d’aide à la déision ELECTRE TRI dans un système d’information géographique open source : Olivier Sobrie, Marc Pirlot, Florent Joerin

### **Procédure pessimiste**

In [16]:
def PessimisticmajoritySorting(categories : list, aliments : dict, profils : dict, indices_de_concordance_globaux : dict, seuil_de_majorite : float) :
  """ Classe chaque aliment dans une catégorie selon la procédure d'affectation pessimiste
      Parametres : 
          - categories (list) : liste des categories 
              Exemple : categories= ["A", "B", "C", "D", "E"]
          - aliments (dict) : dictionnaire conténant les aliments (la clé correspond au nom de l'aliment, et la valeur aux 
                              évaluation qualitative attribuée à l'aliment pour chaque critères)
                      Exemple : aliments = {"aliment1" : [1, 2, 3, 1, 0.1, 10], 'aliment2': [2, 0, 1, 5, 1, 10]}
          - proids (liste) : listes de poids pour chaque critères (poids[j] = poids du critère j)
                      Exemple : poids = [1, 1, 1, 1, 2, 2]
          - indices_de_concordance_globaux (dict) : dictionnaires conténant les indices de concordance globaux calculer comme illustrer à l'étapes 2
          - seuil_de_majorite (float) : réel entre 0 et 1 (ou 0% à 100%) répresentant le seuil de majorité
  """
  result = {}
  r = len(categories)
  b = list(profils.keys())
  for H in aliments.keys() :
    for k in range(r-1, -1, -1) : 
      H_S_b_k, _ = surclass(seuil_de_majorite, H, b[k], indices_de_concordance_globaux)
      if H_S_b_k :
        result[H] = categories[k]
        break
    result[H] = result.get(H, categories[0]) 
  return result

### **Procédure optimiste**

In [26]:
def OptimisticmajoritySorting(categories : list, aliments : dict, profils : dict, indices_de_concordance_globaux : dict, seuil_de_majorite : float) : 
  """ Classe chaque aliment dans une catégorie selon la procédure d'affectation optimiste
      Parametres : 
          - categories (list) : liste des categories 
              Exemple : categories= ["A", "B", "C", "D", "E"]
          - aliments (dict) : dictionnaire conténant les aliments (la clé correspond au nom de l'aliment, et la valeur aux 
                              évaluation qualitative attribuée à l'aliment pour chaque critères)
                      Exemple : aliments = {"aliment1" : [1, 2, 3, 1, 0.1, 10], 'aliment2': [2, 0, 1, 5, 1, 10]}
          - proids (liste) : listes de poids pour chaque critères (poids[j] = poids du critère j)
                      Exemple : poids = [1, 1, 1, 1, 2, 2]
          - indices_de_concordance_globaux (dict) : dictionnaires conténant les indices de concordance globaux calculer comme illustrer à l'étapes 2
          - seuil_de_majorite (float) : réel entre 0 et 1 (ou 0% à 100%) répresentant le seuil de majorité
  """
  result = {}
  r = len(categories)
  b = list(profils.keys())
  for H in aliments.keys() :
    for k in range(1, r+1) :
      H_S_b_k, b_k_S_H = surclass(seuil_de_majorite, H, b[k], indices_de_concordance_globaux)
      #if b_k_S_H :
      if b_k_S_H and not H_S_b_k :
        result[H] = categories[k-1]
        break
    result[H] = result.get(H, categories[r-1])
  return result

## **Fonction utiles pour la lectures et l'écriture dans les fichiers excels**

In [60]:
def get_profils(file_path : str, sheet_name : str = None):
    """Prend un fichier excel conténant les profils ou les aliments et retourne le résultats au format :
      - aliments = {"aliment_1" : [1, 2, 3, 1, 0.1, 10], ..., 'aliment_n': [2, 0, 1, 5, 1, 10]} par exemple, s'il s'agit des aliments
      - profils = {"b6" : [100, 0, 0, 0, 100, 100], ..., "b1" : [10000, 100, 100, 100, 0, 0]} par exemple, s'il s'agit des profils

      Parametres :
        - file_path (str) : chemin du fichier excel
        - sheet_name (str) nom de la feuille excel cible (si aucune feuille n'est spécifiée, la prémiere feuille est considérée)
    """
    if sheet_name is None :
        content = pd.read_excel(file_path, index_col=0)
    else :
        content = pd.read_excel(file_path, index_col=0, sheet_name=sheet_name)
    indexs = content.index
    profils = {key : [] for key in indexs}
    for critere in content.keys() :
        C = content[critere]
        for key in indexs :
            profils[key].append(C[key])
    return profils

def get_criteres_poids(file_path : str, sheet_name : str = None):
    """Prend un fichier excel conténant les criteres, leur poids et leur type (max ou min) et retourne le résultats au format :
      - criteres = [["Énergie", 'min'], ["Acide Gras sat.", 'min'], ["Sucre", 'min'], ["Sodium", 'min'], ["Protéine", 'max'], ["Fibre", 'max']] par exemple
      - poids = [1, 1, 1, 1, 2, 2] par exemple

      Parametres :
        - file_path (str) : chemin du fichier excel
        - sheet_name (str) nom de la feuille excel cible (si aucune feuille n'est spécifiée, la prémiere feuille est considérée)
    """
    if sheet_name is None :
        content = pd.read_excel(file_path, index_col=0)
    else :
        content = pd.read_excel(file_path, index_col=0, sheet_name=sheet_name)
    #indexs = content.index
    poids, criteres = [], []
    for critere in content.keys() :
        C = content[critere]
        poids.append(C["poids"])
        criteres.append([critere, C["type_critere"]])
    return criteres, poids

def to_excel(classement, output_file : str, sheet_name : str = None):
    """Fonction écrivant les résultats fournis par les procédures d'affectation dans les fichiers excel

      Parametres :
        - classement (dict) : dictionnaire conténant pour chaque aliment (clé = nom de l'aliment) sa catégorie (valeur)
        - output_file (str) : chemin du fichier excel dans lequel le résultat sera stocké
        - sheet_name (str) nom de la feuille excel cible (si aucune feuille n'est spécifié, la prémiere feuille est considérée)
    """
    index = classement.keys()
    columns = ["categories"]
    df = list(classement.values())
    df = [[a] for a in df]
    df = pd.DataFrame(df, index=index, columns = columns)
    if sheet_name is None :
        df.to_excel(output_file)
    else :
        df.to_excel(output_file, sheet_name = sheet_name)

# **Test des fonctions par simultation sur des données fictives**

In [28]:
get_c_j(H = [2, 1, 1, 100, 1, 1], b_i = [2, 1, 1, 100, 1, 1], j = 5, type_critere = "max")

(1, 1)

In [38]:
categories = ["A", "B", "C", "D", "E"]
criteres = [["Énergie", 'min'], ["Acide Gras sat.", 'min'], ["Sucre", 'min'], ["Sodium", 'min'], ["Protéine", 'max'], ["Fibre", 'max']]
poids = [1, 1, 1, 1, 2, 2]
profils = {"b6" : [100, 0, 0, 0, 100, 100], "b5" : [1550, 11, 0.8, 0.3, 10, 11], "b4" : [1650, 14, 1, 0.4, 7, 8], "b3" : [1750, 17, 1.7, 0.5, 4, 5], "b2" : [1850, 20, 4, 0.6, 3, 2.5], "b1" : [10000, 100, 100, 100, 0, 0]}
aliments = {"aliment_1" : [1.5, 14, 3, 2, 0, 1], 'aliment_2': [0, 2, 1, 1, 10.11, 4]}
seuil_de_majorite = 0.5

## **Étape 1 : Détermination des indices de concordance partiels**

In [39]:
c = get_indices_de_concordance_partiels(criteres = criteres, aliments = aliments, profils = profils)
c

{0: {'aliment_1': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1},
  'aliment_2': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1},
  'b1': {'aliment_1': 0, 'aliment_2': 0},
  'b2': {'aliment_1': 0, 'aliment_2': 0},
  'b3': {'aliment_1': 0, 'aliment_2': 0},
  'b4': {'aliment_1': 0, 'aliment_2': 0},
  'b5': {'aliment_1': 0, 'aliment_2': 0},
  'b6': {'aliment_1': 0, 'aliment_2': 0}},
 1: {'aliment_1': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 0, 'b6': 0},
  'aliment_2': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 0},
  'b1': {'aliment_1': 0, 'aliment_2': 0},
  'b2': {'aliment_1': 0, 'aliment_2': 0},
  'b3': {'aliment_1': 0, 'aliment_2': 0},
  'b4': {'aliment_1': 1, 'aliment_2': 0},
  'b5': {'aliment_1': 1, 'aliment_2': 0},
  'b6': {'aliment_1': 1, 'aliment_2': 1}},
 2: {'aliment_1': {'b1': 1, 'b2': 1, 'b3': 0, 'b4': 0, 'b5': 0, 'b6': 0},
  'aliment_2': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 0, 'b6': 0},
  'b1': {'aliment_1': 0, 'aliment_2': 0},
  'b2': {'aliment

## **Étape 2 : Détermination des indices de concordance globaux**

In [40]:
C =  get_indices_de_concordance_globaux(n = len(criteres), indices_de_concordance_partiels = c, proids = poids)
C

{'aliment_1': {'b1': 1.0,
  'b2': 0.375,
  'b3': 0.25,
  'b4': 0.25,
  'b5': 0.125,
  'b6': 0.125},
 'aliment_2': {'b1': 1.0,
  'b2': 0.875,
  'b3': 0.625,
  'b4': 0.625,
  'b5': 0.5,
  'b6': 0.125},
 'b1': {'aliment_1': 0.25, 'aliment_2': 0.0},
 'b2': {'aliment_1': 0.625, 'aliment_2': 0.125},
 'b3': {'aliment_1': 0.75, 'aliment_2': 0.375},
 'b4': {'aliment_1': 0.875, 'aliment_2': 0.5},
 'b5': {'aliment_1': 0.875, 'aliment_2': 0.5},
 'b6': {'aliment_1': 0.875, 'aliment_2': 0.875}}

## **Étape 3 & 4 : Relation de surclassement & Procédures d’affectation**

### **1) pessimiste**

In [41]:
categotiries_pessimist = PessimisticmajoritySorting(categories = categories, aliments = aliments, profils = profils, indices_de_concordance_globaux = C, seuil_de_majorite = seuil_de_majorite) 
categotiries_pessimist

{'aliment_1': 'A', 'aliment_2': 'E'}

### **1) optimiste**

In [42]:
categotiries_optimist = OptimisticmajoritySorting(categories = categories, aliments = aliments, profils = profils, indices_de_concordance_globaux = C, seuil_de_majorite = seuil_de_majorite)
categotiries_optimist

{'aliment_1': 'A', 'aliment_2': 'E'}

# **Test sur des données venants des fichiers**

**Utiliser répétitivement et dans l'ordre ces cellules pour générer progressivement les résultats.** 

C'est-à-dire : 
* specifier les chemins de fichiers
* exécuter les cellules jusqu'à obtention et ecritures des résultats dans le fichier de sortie
* Recommencer pour un (d') autre(s) fichier(s) 



> `NB` : Spécifier les chemins vers vos fichiers et s'assuser que les fichiers existent et respectent le format réquis par les fonctions de lecture (des templates sont présent dans ce [dossier google drive](https://https://drive.google.com/drive/folders/1rvEylqVGC4AGGeJzFpxWlpNaI3H_uUXi?usp=sharing)).

> Si le fichier est mal encodé (avec les symboles bizarres au niveau des noms des aliments par exemple), la lecture va générer une erreur. 

> Dans mon cas j'ai upload les fichiers sous google colab (Voir l'icone en haut à gauche, portant une flèche montante)



> Apres exécutions j'ai télécharger les fichiers ```output_optimiste.xlsx``` et ```output_pesimite.xlsx```







In [95]:
profils_file = "profils.xlsx"
criteres_file = "criteres.xlsx"
aliments_file = "aliments.xlsx" # "BD3_brut.xlsx" pour la vrai BD
output_file_pesimite =  "output_pesimite.xlsx"
output_file_optimiste =  "output_optimiste.xlsx"
seuil_de_majorite = 0.5

In [96]:
criteres, poids = get_criteres_poids(file_path = criteres_file)
criteres, poids

([['energy100g', 'min'],
  ['saturatedfat100g', 'min'],
  ['sugars100g', 'min'],
  ['fiber100g', 'min'],
  ['proteins100g', 'max'],
  ['sodium100g', 'max']],
 [1, 1, 1, 1, 2, 2])

In [97]:
profils = get_profils(file_path = profils_file)
profils

{'b1': [10000, 100, 100.0, 0.0, 0, 100.0],
 'b2': [1850, 20, 4.0, 2.5, 3, 0.6],
 'b3': [1750, 17, 1.7, 5.0, 4, 0.5],
 'b4': [1650, 14, 1.0, 8.0, 7, 0.4],
 'b5': [1550, 11, 0.8, 11.0, 10, 0.3],
 'b6': [100, 0, 0.0, 100.0, 100, 0.0]}

In [98]:
aliments = get_profils(file_path = aliments_file)
aliments

{'bongo': [0, 100, 0, 0, 0, 0],
 'dumba': [100, 0, 0, 0, 0, 0],
 'eru': [0, 0, 0, 0, 0, 100],
 'koki': [0, 0, 100, 0, 0, 0],
 'okok': [0, 0, 0, 0, 100, 0],
 'taro': [0, 0, 0, 100, 0, 0]}

### Optionnel : on ordonne les profils et les catégories
* se rassurer qu'on part de $b_6$ à $b_1$ pour les profils
* se rassurer qu'on part de $A$ à $E$ pour les catégories

In [99]:
profils = {k : v for k, v in sorted(profils.items(), key=lambda b : b[0], reverse = True)} # se rassurer qu'on part de b6 à b1
categories = sorted(categories, key = lambda c : c, reverse = False) # se rassurer qu'on part de A à E

## **Étape 1 : Détermination des indices de concordance partiels**

In [100]:
c = get_indices_de_concordance_partiels(criteres = criteres, aliments = aliments, profils = profils)
c

{0: {'b1': {'bongo': 0, 'dumba': 0, 'eru': 0, 'koki': 0, 'okok': 0, 'taro': 0},
  'b2': {'bongo': 0, 'dumba': 0, 'eru': 0, 'koki': 0, 'okok': 0, 'taro': 0},
  'b3': {'bongo': 0, 'dumba': 0, 'eru': 0, 'koki': 0, 'okok': 0, 'taro': 0},
  'b4': {'bongo': 0, 'dumba': 0, 'eru': 0, 'koki': 0, 'okok': 0, 'taro': 0},
  'b5': {'bongo': 0, 'dumba': 0, 'eru': 0, 'koki': 0, 'okok': 0, 'taro': 0},
  'b6': {'bongo': 0, 'dumba': 1, 'eru': 0, 'koki': 0, 'okok': 0, 'taro': 0},
  'bongo': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1},
  'dumba': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1},
  'eru': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1},
  'koki': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1},
  'okok': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1},
  'taro': {'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1}},
 1: {'b1': {'bongo': 1, 'dumba': 0, 'eru': 0, 'koki': 0, 'okok': 0, 'taro': 0},
  'b2': {'bongo': 1, 'dumba': 0, 'eru': 0, 'koki': 0, 'ok

## **Étape 2 : Détermination des indices de concordance globaux**

In [101]:
C =  get_indices_de_concordance_globaux(n = len(criteres), indices_de_concordance_partiels = c, proids = poids)
C

{'b1': {'bongo': 0.75,
  'dumba': 0.625,
  'eru': 0.625,
  'koki': 0.75,
  'okok': 0.375,
  'taro': 0.625},
 'b2': {'bongo': 0.625,
  'dumba': 0.5,
  'eru': 0.25,
  'koki': 0.625,
  'okok': 0.25,
  'taro': 0.625},
 'b3': {'bongo': 0.625,
  'dumba': 0.5,
  'eru': 0.25,
  'koki': 0.625,
  'okok': 0.25,
  'taro': 0.625},
 'b4': {'bongo': 0.625,
  'dumba': 0.5,
  'eru': 0.25,
  'koki': 0.625,
  'okok': 0.25,
  'taro': 0.625},
 'b5': {'bongo': 0.625,
  'dumba': 0.5,
  'eru': 0.25,
  'koki': 0.625,
  'okok': 0.25,
  'taro': 0.625},
 'b6': {'bongo': 0.75,
  'dumba': 0.875,
  'eru': 0.5,
  'koki': 0.75,
  'okok': 0.75,
  'taro': 0.875},
 'bongo': {'b1': 0.75,
  'b2': 0.375,
  'b3': 0.375,
  'b4': 0.375,
  'b5': 0.375,
  'b6': 0.625},
 'dumba': {'b1': 0.75, 'b2': 0.5, 'b3': 0.5, 'b4': 0.5, 'b5': 0.5, 'b6': 0.75},
 'eru': {'b1': 1.0,
  'b2': 0.75,
  'b3': 0.75,
  'b4': 0.75,
  'b5': 0.75,
  'b6': 0.75},
 'koki': {'b1': 0.75,
  'b2': 0.375,
  'b3': 0.375,
  'b4': 0.375,
  'b5': 0.375,
  'b6': 0.6

### Étape 3 & 4 : Relation de surclassement & Procédures d’affectation

### **1) pessimiste**

In [102]:
categotiries_pessimist = PessimisticmajoritySorting(categories = categories, aliments = aliments, profils = profils, indices_de_concordance_globaux = C, seuil_de_majorite = seuil_de_majorite) 
categotiries_pessimist

{'bongo': 'A', 'dumba': 'E', 'eru': 'E', 'koki': 'A', 'okok': 'E', 'taro': 'A'}

**Ecriture du résultat dans le fichier de sortie, sur la feuille ```categotiries_pessimistes```**

In [103]:
to_excel(categotiries_pessimist, output_file_pesimite, "categotiries_pessimistes")

### **2) Optimiste**

In [104]:
categotiries_optimist = OptimisticmajoritySorting(categories = categories, aliments = aliments, profils = profils, indices_de_concordance_globaux = C, seuil_de_majorite = seuil_de_majorite)

In [105]:
categotiries_optimist

{'bongo': 'A', 'dumba': 'E', 'eru': 'E', 'koki': 'A', 'okok': 'E', 'taro': 'A'}

**Ecriture du résultat dans le fichier de sortie, sur la feuille ```categotiries_optimistes```**

In [106]:
to_excel(categotiries_optimist, output_file_optimiste, "categotiries_optimistes")